<a href="https://colab.research.google.com/github/mleyvaz/AV/blob/master/Copy_of_LLM_fsQCA_GuayaquilNEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM → fsQCA (MVP)
## Análisis de explicaciones causales de la violencia en Guayaquil

Este cuaderno:
- Descarga noticias
- Extrae causas con LLMs
- Construye casos fsQCA
- Calcula consistencia y cobertura


In [ ]:
!pip -q install openai pandas numpy trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 9.1 MB/s eta 0:00:00


In [ ]:
import os, json, re, time, getpass
import numpy as np
import pandas as pd
import trafilatura
from openai import OpenAI

print('Librerías cargadas correctamente')

Librerías cargadas correctamente


## Configuración de OpenAI API Key

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('Pega tu OPENAI API KEY: ')
client = OpenAI()
MODEL = 'gpt-4o-mini'
print('Cliente OpenAI listo')

Pega tu OPENAI API KEY: ··········
Cliente OpenAI listo


## Variables causales y outcome

In [ ]:
VARIABLES = {
 'A_gang_fragmentation': 'Fragmentación o alianzas entre bandas',
 'B_drug_routes_ports': 'Control de rutas del narcotráfico/puertos',
 'C_extortion_vacunas': 'Extorsión y vacunas',
 'D_territorial_war': 'Guerra territorial',
 'E_weapons_traffic': 'Tráfico de armas',
 'F_prison_link': 'Vínculo con cárceles',
 'G_institutional_weakness': 'Debilidad institucional',
 'H_economic_pressure': 'Presión económica'
}

OUTCOME_KEY = 'Y_violence_spike'
print('Variables definidas')

Variables definidas


## URLs de noticias (puedes ampliar la lista)

In [ ]:
URLS = [
  # Bandas, alianzas, disputas territoriales
  "https://www.swissinfo.ch/spa/traiciones-y-nuevas-alianzas-entre-bandas-criminales-recrudecen-la-violencia-en-guayaquil/89498744",
  "https://www.reuters.com/world/americas/ecuadors-crackdown-gangs-fractures-criminal-networks-fuels-bloodshed-2025-10-29/",
  "https://elpais.com/america/2025-03-07/asesinadas-22-personas-en-una-zona-urbana-de-guayaquil-en-una-escalada-sin-freno-de-la-violencia-en-ecuador.html",
  "https://www.eluniverso.com/noticias/informes/la-violencia-en-ecuador-no-da-tregua-nota/",
  "https://www.expreso.ec/actualidad/hay-mas-homicidios-en-ecuador-pese-operativos-de-seguridad-268021.html",

  # Extorsión, “vacunas” y delincuencia urbana
  "https://www.primicias.ec/seguridad/extorsion-guayaquil-tenderos-amenazados-vacunas-extorsivas-secuestro-comerciantes-98503/",
  "https://www.elcomercio.com/actualidad/seguridad/extorsion-vacunas-ciudades-amenazas-comerciantes.html",
  "https://www.oeco.padf.org/wp-content/uploads/2024/05/OECO_Radiografia-de-la-extorsion-en-Ecuador-1.pdf",
  "https://www.oeco.padf.org/wp-content/uploads/2023/09/Caracterizacion-Crimen-Organizado-Version-corta-V2.pdf",

  # Homicidios / patrones territoriales
  "https://www.primicias.ec/guayaquil/tres-distritos-guayaquil-suman-52-homicidios-son-barrios-que-concentran-violencia-110953/",
  "https://oeco.padf.org/boletin-semestral-de-homicidios-intencionales-en-ecuador/",
  "https://www.ecuavisa.com/noticias/ecuador/violencia-inseguridad-impulsan-desplazamiento-forzado-familias-ecuador-HM10587145",
  "https://www.elcomercio.com/actualidad/seguridad/homicidios-intencionales-aumentan-ecuador-2025/",
  "https://www.france24.com/es/programas/reporteros/20240517-guayaquil-una-ciudad-ecuatoriana-acorralada-por-el-crimen-organizado",

  # Crimen organizado / análisis experto
  "https://www.crisisgroup.org/latin-america-caribbean/ecuador/109-paradise-lost-ecuadors-battle-organised-crime",
  "https://globalinitiative.net/analysis/ecuadors-criminal-crisis/",
  "https://globalinitiative.net/analysis/ecuador-gangs-drugs-terrorism-violence/",
  "https://www.bbc.com/news/world-latin-america-66250716",
  "https://www.bbc.com/mundo/noticias-america-latina-64486858",

  # Política pública / respuesta del Estado
  "https://www.eluniverso.com/noticias/politica/operativos-seguridad-ecuador-efectos-violencia-guayaquil-nota/",
  "https://www.expreso.ec/actualidad/analisis-seguridad-ciudadana-ecuador-violencia-criminal-270115.html",
  "https://www.elcomercio.com/opinion/editorial/seguridad-ecuador-politicas-represion.html",
  "https://www.larepublica.ec/blog/2025/09/14/analisis-violencia-seguridad-ciudadana-ecuador/",

  # Casos y eventos extremos
  "https://www.apnews.com/article/b797b631fc6dcce3762b9f4efa685d83",
  "https://www.eluniverso.com/noticias/2025/08/02/nota/1234/explosivo-ataque-guayaquil-criminalidad",
  "https://www.primicias.ec/seguridad/bomba-explosion-guayaquil-barrios/",
  "https://www.elcomercio.com/actualidad/violencia-explosivos-ciudades-ecuador.html",

  # Opinión y análisis social
  "https://www.larepublica.ec/analisis/violencia-ciudadana-guayaquil-sociedad/",
  "https://www.primicias.ec/opinion/cultura-violencia-ecuador-guayaquil/",
  "https://www.elcomercio.com/opinion/seguridad-ciudadana-mirada-social.html",
  "https://www.eluniverso.com/opinion/articulo-violencia-y-sociedad-guayaquil/",
  "https://www.expreso.ec/opinion/comunidad-violencia-seguridad-ciudadana.html",

  # Estudios sobre extorsión, bandas y respuestas comunitarias
  "https://www.researchgate.net/publication/XXX_violencia_y_extorsion_UrbanStudies_Guayaquil",  # ejemplo académico general
  "https://www.universidad.edu.ec/tesis/analisis-criminalidad-guayaquil.pdf",          # ejemplo de tesis
  "https://repositorio.universidad.edu.ec/bitstream/handle/XXX-homicidios-guayaquil.pdf"  # otro
]

print("URLs cargadas:", len(URLS))


URLs cargadas: 35


## Descarga y extracción de texto

In [ ]:
def fetch_text(url):
 data = trafilatura.fetch_url(url)
 return trafilatura.extract(data) if data else ''

docs = []
for i, url in enumerate(URLS, start=1):
 txt = fetch_text(url)
 docs.append({'case_id': f'N{i:03d}', 'url': url, 'text': txt[:8000]})

print('Noticias procesadas:', len(docs))

ERROR:trafilatura.downloads:not a 200 response: 401 for URL https://www.reuters.com/world/americas/ecuadors-crackdown-gangs-fractures-criminal-networks-fuels-bloodshed-2025-10-29/
ERROR:trafilatura.downloads:not a 200 response: 403 for URL https://www.expreso.ec/actualidad/hay-mas-homicidios-en-ecuador-pese-operativos-de-seguridad-268021.html
ERROR:trafilatura.downloads:not a 200 response: 403 for URL https://www.primicias.ec/seguridad/extorsion-guayaquil-tenderos-amenazados-vacunas-extorsivas-secuestro-comerciantes-98503/
ERROR:trafilatura.downloads:not a 200 response: 404 for URL https://www.elcomercio.com/actualidad/seguridad/extorsion-vacunas-ciudades-amenazas-comerciantes.html
ERROR:trafilatura.downloads:download error: https://www.oeco.padf.org/wp-content/uploads/2024/05/OECO_Radiografia-de-la-extorsion-en-Ecuador-1.pdf HTTPSConnectionPool(host='www.oeco.padf.org', port=443): Max retries exceeded with url: /wp-content/uploads/2024/05/OECO_Radiografia-de-la-extorsion-en-Ecuador-1.

Noticias procesadas: 35


In [ ]:
COND_KEYS = list(VARIABLES.keys())

def prompt_extract_claims(text: str) -> str:
    vars_txt = "\n".join([f"- {k}: {v}" for k,v in VARIABLES.items()])
    return f"""
Return ONLY a valid JSON array. No markdown.

Extract causal claims that explain spikes of violence in Guayaquil.
Use ONLY these condition variables and ONLY presence/absence logic.

CONDITIONS:
{vars_txt}

OUTCOME:
- {OUTCOME_KEY}: Spike/increase in violence/homicides in Guayaquil

Rules:
- Each claim must use 1-4 conditions.
- polarity: "+" (presence) or "-" (absence).
- evidence_span <= 25 words.
- confidence_textual in [0,1].

Output JSON array with objects:
{{
  "claim_id": "C001",
  "outcome": "{OUTCOME_KEY}",
  "direction": "sufficient",
  "conditions": [{{"var":"A_gang_fragmentation","polarity":"+","evidence_span":"..."}}],
  "confidence_textual": 0.0
}}

TEXT:
<<<{text}>>>
""".strip()

def prompt_build_case(case_id: str, text: str) -> str:
    vars_txt = "\n".join([f"- {k}: {v}" for k,v in VARIABLES.items()])
    keys = COND_KEYS + [OUTCOME_KEY]
    return f"""
Return ONLY valid JSON. No markdown.

For this news text, assign 1 (present) or 0 (absent) to each variable.

CONDITIONS:
{vars_txt}

OUTCOME:
- {OUTCOME_KEY}: Spike/increase in violence/homicides in Guayaquil

Rules:
- Use 1 only if there is explicit or strongly implied evidence.
- Use 0 otherwise.
- Provide evidence spans only for variables set to 1.

Return format:
{{
  "case_id": "{case_id}",
  {", ".join([f'"{k}": 0' for k in keys])},
  "evidence": {{}}
}}

TEXT:
<<<{text}>>>
""".strip()

print("Prompts listos.")


Prompts listos.


## Fin del MVP

A partir de aquí puedes extender a:
- fuzzy fsQCA
- negación (~A)
- score neutrosófico (T,I,F)


In [ ]:
def call_llm_json(prompt: str, model: str = MODEL) -> str:
    r = client.responses.create(
        model=model,
        input=prompt,
        text={"format": {"type": "json_object"}}  # <- CAMBIO AQUÍ
    )
    return r.output_text


In [ ]:
def safe_json_loads(s: str):
    # Si el modelo devuelve JSON imperfecto, esto fallará.
    # (Si te falla, te doy una celda de "repair" para arreglarlo automáticamente.)
    return json.loads(s)

all_cases = []
all_claims = []

for d in docs:
    case_id = d["case_id"]
    text = d["text"]

    # 1) Caso QCA (0/1)
    case_raw = call_llm_json(prompt_build_case(case_id, text))
    case_obj = safe_json_loads(case_raw)
    case_obj["url"] = d["url"]
    all_cases.append(case_obj)

    # 2) Claims causales (JSON array)
# Claims
claims_raw = call_llm_json(prompt_extract_claims(text))
claims_obj = json.loads(claims_raw)

# Aseguramos que sea una lista
claims = claims_obj.get("claims", [])
assert isinstance(claims, list), "claims no es una lista"

for i, c in enumerate(claims, start=1):
    assert isinstance(c, dict), f"claim no es dict: {c}"
    c = dict(c)  # ahora sí es seguro

    c["global_claim_id"] = f"{case_id}::{c.get('claim_id', f'C{i:03d}')}"
    c["source_case_id"] = case_id
    all_claims.append(c)


cases_df = pd.DataFrame(all_cases)
print("Casos:", len(cases_df), "Claims:", len(all_claims))
cases_df.head()


Casos: 35 Claims: 4


,case_id,A_gang_fragmentation,B_drug_routes_ports,C_extortion_vacunas,D_territorial_war,E_weapons_traffic,F_prison_link,G_institutional_weakness,H_economic_pressure,Y_violence_spike,evidence,url
0,N001,1,1,1,0,1,0,1,1,1,{'A_gang_fragmentation': 'El coronel Víctor Or...,https://www.swissinfo.ch/spa/traiciones-y-nuev...
1,N002,0,0,0,0,0,0,0,0,0,{},https://www.reuters.com/world/americas/ecuador...
2,N003,1,0,1,1,1,0,1,0,1,{'A_gang_fragmentation': 'la banda criminal qu...,https://elpais.com/america/2025-03-07/asesinad...
3,N004,1,0,1,1,1,1,1,0,1,{'A_gang_fragmentation': '7.650 muertes corres...,https://www.eluniverso.com/noticias/informes/l...
4,N005,0,0,0,0,0,0,0,0,0,{},https://www.expreso.ec/actualidad/hay-mas-homi...


In [ ]:
os.makedirs("outputs", exist_ok=True)

cases_df.to_csv("outputs/cases.csv", index=False)

with open("outputs/claims.jsonl", "w", encoding="utf-8") as f:
    for c in all_claims:
        f.write(json.dumps(c, ensure_ascii=False) + "\n")

print("Guardado: outputs/cases.csv y outputs/claims.jsonl")


Guardado: outputs/cases.csv y outputs/claims.jsonl


In [ ]:
def build_truth_table(df: pd.DataFrame, conds, outcome):
    tt = (df.groupby(conds, as_index=False)
            .agg(n_cases=("case_id","count"),
                 y_rate=(outcome,"mean"))
            .sort_values(["n_cases","y_rate"], ascending=False))
    tt["row_consistency"] = tt["y_rate"]  # crisp
    return tt

tt = build_truth_table(cases_df, COND_KEYS, OUTCOME_KEY)
tt.to_csv("outputs/truth_table.csv", index=False)

tt.head(25)


,A_gang_fragmentation,B_drug_routes_ports,C_extortion_vacunas,D_territorial_war,E_weapons_traffic,F_prison_link,G_institutional_weakness,H_economic_pressure,n_cases,y_rate,row_consistency
0,0,0,0,0,0,0,0,0,26,0.0,0.0
1,1,0,0,0,0,0,0,0,1,1.0,1.0
2,1,0,1,1,1,0,1,0,1,1.0,1.0
3,1,0,1,1,1,1,1,0,1,1.0,1.0
4,1,1,0,0,0,0,1,0,1,1.0,1.0
5,1,1,0,1,0,0,0,0,1,1.0,1.0
6,1,1,0,1,1,1,1,0,1,1.0,1.0
7,1,1,1,0,1,0,1,1,1,1.0,1.0
8,1,1,1,1,0,1,1,0,1,1.0,1.0
9,1,1,1,1,1,0,1,1,1,1.0,1.0


In [ ]:
import numpy as np

def config_membership_crisp(df: pd.DataFrame, config):
    return df[config].min(axis=1).to_numpy()

def consistency_coverage(df: pd.DataFrame, config, outcome):
    X = config_membership_crisp(df, config)
    Y = df[outcome].to_numpy()
    num = np.minimum(X, Y).sum()
    den_cons = X.sum() if X.sum() > 0 else 1e-9
    den_cov  = Y.sum() if Y.sum() > 0 else 1e-9
    return float(num/den_cons), float(num/den_cov), float(X.sum())

def claim_to_config(claim):
    # MVP: solo condiciones con presencia '+'
    cfg = [c["var"] for c in claim.get("conditions", []) if c.get("polarity","+") == "+"]
    return sorted(list(set(cfg)))

scores = []
for cl in all_claims:
    cfg = claim_to_config(cl)
    if not cfg:
        continue

    cons, cov, supp = consistency_coverage(cases_df, cfg, OUTCOME_KEY)

    X = config_membership_crisp(cases_df, cfg)
    Y = cases_df[OUTCOME_KEY].to_numpy()
    contradictions = float(((X > 0.5) & (Y < 0.5)).sum())
    contrad_rate = contradictions / (supp if supp > 0 else 1e-9)

    scores.append({
        "global_claim_id": cl["global_claim_id"],
        "source_case_id": cl["source_case_id"],
        "config": "*".join(cfg),
        "consistency": cons,
        "coverage": cov,
        "support": supp,
        "contradiction_rate": contrad_rate,
        "confidence_textual": cl.get("confidence_textual", None),
    })

scores_df = pd.DataFrame(scores).sort_values(
    ["consistency","coverage","support"],
    ascending=False
)

scores_df.to_csv("outputs/claim_scores.csv", index=False)
scores_df.head(30)


,global_claim_id,source_case_id,config,consistency,coverage,support,contradiction_rate,confidence_textual
0,N035::C001,N035,A_gang_fragmentation*B_drug_routes_ports,1.0,0.666667,6.0,0.0,0.8
2,N035::C003,N035,E_weapons_traffic*G_institutional_weakness,1.0,0.555556,5.0,0.0,0.6
1,N035::C002,N035,C_extortion_vacunas*D_territorial_war,1.0,0.444444,4.0,0.0,0.7
3,N035::C004,N035,F_prison_link*H_economic_pressure,0.0,0.000000,0.0,0.0,0.5


In [ ]:
!apt-get -qq update
!apt-get -qq install -y r-base
!R -q -e "install.packages('QCA', repos='https://cloud.r-project.org')"
print("R + QCA instalado")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
> install.packages('QCA', repos='https://cloud.r-project.org')
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘admisc’, ‘declared’, ‘venn’, ‘lpSolve’

trying URL 'https://cloud.r-project.org/src/contrib/admisc_0.39.tar.gz'
trying URL 'https://cloud.r-project.org/src/contrib/declared_0.25.tar.gz'
trying URL 'https://cloud.r-project.org/src/contrib/venn_1.12.tar.gz'
trying URL 'https://cloud.r-project.org/src/contrib/lpSolve_5.6.23.tar.gz'
trying URL 'https://cloud.r-project.org/src/contrib/QCA_3.23.tar.gz'
* installing *source* package ‘admisc’ ...
** this is package ‘admisc’ version ‘0.39’
** package ‘admisc’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C compiler: ‘gcc (Ubuntu 11.4.0-1ubuntu1

In [ ]:
cases_df[["case_id"] + COND_KEYS + [OUTCOME_KEY]].to_csv("outputs/cases_for_R.csv", index=False)


In [ ]:
%%bash
R --vanilla -q <<'RSCRIPT'
library(QCA)

df <- read.csv("outputs/cases_for_R.csv", stringsAsFactors=FALSE)

# Condiciones = todas menos case_id y outcome
conds <- names(df)[!(names(df) %in% c("case_id","Y_violence_spike"))]

tt <- truthTable(
  df,
  outcome="Y_violence_spike",
  conditions=conds,
  incl.cut=0.8,
  show.cases=TRUE
)
print(tt)

# Minimización (parsimoniosa)
sol <- eqmcc(tt, include="?", details=TRUE)
print(sol)

RSCRIPT


> library(QCA)
> 
> df <- read.csv("outputs/cases_for_R.csv", stringsAsFactors=FALSE)
> 
> # Condiciones = todas menos case_id y outcome
> conds <- names(df)[!(names(df) %in% c("case_id","Y_violence_spike"))]
> 
> tt <- truthTable(
+   df,
+   outcome="Y_violence_spike",
+   conditions=conds,
+   incl.cut=0.8,
+   show.cases=TRUE
+ )
> print(tt)

  OUT: output value
    n: number of cases in configuration
 incl: sufficiency inclusion score
  PRI: proportional reduction in inconsistency

      A_gang_fragmentation B_drug_routes_ports C_extortion_vacunas
  1            0                    0                   0         
129            1                    0                   0         
187            1                    0                   1         
191            1                    0                   1         
195            1                    1                   0         
209            1                    1                   0         
223            1                    1  

Loading required package: admisc

To cite package QCA in publications, please use:
  Dusa, Adrian (2019) QCA with R. A Comprehensive Resource.
  Springer International Publishing.

To run the graphical user interface, use: runGUI()


Warning message:
In eqmcc(tt, include = "?", details = TRUE) :
  Function eqmcc() is deprecated, and has been renamed to minimize()

